# TCP编程

## 客户端

大多数连接都是可靠的TCP连接。创建TCP连接时，主动发起连接的叫客户端，被动响应连接的叫服务器。

举个例子，当我们在浏览器中访问新浪时，我们自己的计算机就是客户端，浏览器会主动向新浪的服务器发起连接。如果一切顺利，新浪的服务器接受了我们的连接，一个TCP连接就建立起来的，后面的通信就是发送网页内容了。

所以，我们要创建一个基于TCP连接的Socket，可以这样做

In [4]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('www.sina.com.cn', 80)) # (())

In [5]:
s.send(b'GET /HTTP/1.1\r\nHost:www.sina.com.cn\r\nConnection:close\r\n\r\n') 
# 返回的值是？

57

In [6]:
buffer = []
while True:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break

data = b''.join(buffer)

In [7]:
s.close()

In [8]:
data # 请求回来的数据不对

b'<HTML>\n<HEAD>\n<TITLE>Not Found on Accelerator</TITLE>\n</HEAD>\n\n<BODY BGCOLOR="white" FGCOLOR="black">\n<H1>Not Found on Accelerator</H1>\n<HR>\n\n<FONT FACE="Helvetica,Arial"><B>\nDescription: Your request on the specified host was not found.\nCheck the location and try again.\n</B></FONT>\n<HR>\n</BODY>\n'

In [9]:
header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
with open('sina.htm', 'wb') as f:
    f.write(html)

ValueError: not enough values to unpack (expected 2, got 1)

## 服务器

In [10]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

我们要绑定监听的地址和端口。服务器可能有多块网卡，可以绑定到某一块网卡的IP地址上，也可以用0.0.0.0绑定到所有的网络地址，还可以用127.0.0.1绑定到本机地址。127.0.0.1是一个特殊的IP地址，表示本机地址，如果绑定到这个地址，客户端必须同时在本机运行才能连接，也就是说，外部的计算机无法连接进来。

端口号需要预先指定。因为我们写的这个服务不是标准服务，所以用9999这个端口号。请注意，小于1024的端口号必须要有管理员权限才能绑定

In [11]:
s.bind(('127.0.0.1', 9999)) # 监听端口
s.listen(5) #等待连接的最大数量
print('Waiting for connection')

Waiting for connection


接下来，服务器程序通过一个永久循环来接受来自客户端的连接，accept()会等待并返回一个客户端的连接

In [ ]:
while True:
    sock, addr = s.accept() # 接受一个新连接:
    t = threading.Thread(target=tcplink, args=(sock, addr)) # 创建新线程来处理TCP连接:
    t.start()

每个连接都必须创建新线程（或进程）来处理，否则，单线程在处理连接的过程中，无法接受其他客户端的连接

In [ ]:
#echo_server.py

import 
def tcplink(sock, addr):
    print('Accept new connection from %s:%s' % addr)
    sock.send(b'Welcome')
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf-8') == 'exit':
            break
        sock.send(('Hello, %s!' % data.decode('utf-8')).encode('utf-8'))
    sock.close()
    print('Connection from %s:%s closed' % addr)

连接建立后，服务器首先发一条欢迎消息，然后等待客户端数据，并加上Hello再发送给客户端。如果客户端发送了exit字符串，就直接关闭连接。

要测试这个服务器程序，我们还需要编写一个客户端程序：

In [ ]:
#echo_client.py

import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('127.0.0.1', 9999))
print(s.recv(1024).decode('utf-8'))
for data in [b'Michael', b'Tracy', b'Sarah']:
    s.send(data)
    print(s.recv(1024).decode('utf-8'))
s.send(b'exit')
s.close()

用TCP协议进行Socket编程在Python中十分简单，对于客户端，要主动连接服务器的IP和指定端口，对于服务器，要首先监听指定端口，然后，对每一个新的连接，创建一个线程或进程来处理。通常，服务器程序会无限运行下去。

同一个端口，被一个Socket绑定了以后，就不能被别的Socket绑定了

# 21day

In [ ]:
'''
socket服务器建立流程图
s=socket()
s.bind()
s.listen()
c, a = s.accept()
c.recv() or c.send()
s.close()
'''
import socket

host = ''
port = 10888

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((host, port))
s.listen(1)
conn, addr = s.accept()
print('Client\'s Address:', addr)

while True:
    data = conn.recv(1024)
    if not data:
        break
    print('Receive Data:', data.decode('utf-8'))
    conn.send(data)
conn.close()


In [ ]:
'''
socket建立客户端程序
s=socket()
s.connect()
c.recv() or c.send()
s.close()
'''

import socket

host = 'localhost'
port = 10888

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect((host, port))
data = '你好'
while data:
    s.sendall(data.encode('utf-8'))
    data = s.recv(512)
    print('Receive from server:\n', data.decode('utf-8'))
    data = input('please input a info:\n')
s.close()

